# Bayesian Optimization based on GP

In [1]:
import numpy as np
import sklearn.gaussian_process as gp
from scipy.stats import norm
from scipy.optimize import minimize
from sklearn.gaussian_process.kernels import RBF, Matern
import math
import time
from copy import deepcopy
from multiprocessing import Process, Manager
from acquisitions import EI, UCB
from penalizers import LP, HLP
from objectives import branin
from parallel_BO import sample_next_parallel, acq_parallel, sample_callback

![Figure 1-1](img/Capture.png "Figure 1-1")

## Gaussian Process-based Bayesian Optimization, parallel implementation
- HLP and LP penalizers for syncronous and ascyncronous BO
- Locally and globally estimated L-constants

In [9]:
def bayesian_optimization(n_iters, function, bounds, acq_func = UCB, penalizer = LP, local_L = True,
                          X_init = None, n_init = 10, gp_params = None, find_min = True, alpha = 1e-5, epsilon = 1e-7):
    
    
    
    X_tested = []
    y_tested = []
    n_params = bounds.shape[0]
    
    if X_init is None:
        X_init = np.random.uniform(bounds[:,0], bounds[:,1], (n_init, bounds.shape[0]))
    
    for X in X_init:
        X_tested.append(X)
        y_tested.append(function(X))
    
    X_np = np.array(X_tested)
    y_np = np.array(y_tested)
    
    # creating the gaussian process
    if gp_params is not None:
        gaussian_process = gp.GaussianProcessRegressor(**args) 
    
    else:
        kernel = Matern()
        gaussian_process = gp.GaussianProcessRegressor(
            kernel = kernel, alpha = alpha, n_restarts_optimizer = 10, normalize_y = True)
    
    # CHANGES START HERE
    # CHANGES START HERE
    # CHANGES START HERE
    
    
    for n in range(n_iters):

        X_np = np.array(X_tested)
        y_np = np.array(y_tested)
        
        gaussian_process.fit(X_tested, y_tested)
        
        #sampling of next point - can be done with random search (not implemented) or optimization of act_func
        X_next = sample_next_parallel(acq_parallel, deepcopy(gaussian_process), X_eval, y_np, bounds, find_min = True, n_restarts = 10,
                     acq_func = acq_func, penalizer = penalizer, local_L = local_L, X_under_eval = None)
        
        while np.any(np.abs(X_next - X_tested) <= epsilon):
            X_next = sample_next_parallel(acq_parallel, gaussian_process, X_eval, y_np, bounds, find_min = True, n_restarts = 10,
                     acq_func = acq_func, penalizer = penalizer, local_L = local_L, X_under_eval = None)
            
            print(X_next, 'has already been sampled.\nIteration', n)
        
        y_next = function(X_next)
        X_tested.append(X_next)
        y_tested.append(y_next)
        
        # CHANGES END HERE
        # CHANGES END HERE
        # CHANGES END HERE

    return X_tested, y_tested

## Attempting to write logic for parallel - still to go

In [34]:
def bayesian_optimization(n_iters, function, bounds, acq_func = UCB, penalizer = HLP, local_L = True,
                          X_init = None, n_init = 10, gp_params = None, find_min = True, alpha = 1e-5, epsilon = 1e-7):
    
    
    X_tested = []
    y_tested = []
    n_params = bounds.shape[0]
    
    if X_init is None:
        X_init = np.random.uniform(bounds[:,0], bounds[:,1], (n_init, bounds.shape[0]))
    
    for X in X_init:
        X_tested.append(X)
        y_tested.append(function(X))
    
    # creating the gaussian process
    if gp_params is not None:
        gaussian_process = gp.GaussianProcessRegressor(**args) 
    
    else:
        kernel = Matern()
        gaussian_process = gp.GaussianProcessRegressor(
            kernel = kernel, alpha = alpha, n_restarts_optimizer = 10, normalize_y = True)
        gaussian_process.fit(X_tested, y_tested)
        
    if __name__ == '__main__':

        manager = Manager()
        lock = manager.Lock()
        process_list = [None] * n_processes
        # tracks where the current processes are being evaluated
        X_eval = manager.list([[0] * n_params] * n_processes)
        output_X = manager.list(X_tested)
        output_Y = manager.list(y_tested)
       
        # Initializing, iteration counts the iteration number
        iteration = 0
        for i in range(n_processes):
            process_list[i] = Process(target=sample_callback, args = (X_eval, output_X, output_Y, lock, i, function,
                acq_parallel, deepcopy(gaussian_process), np.array(X_eval), np.array(output_y), bounds, find_min,
                acq_func, penalizer, local_L))
            process_list[i].start()
            print(f'Process {i +1} started')
            iteration += 1
        
        while iteration < n_iters:
            
            # if the process is done
            for i in range(n_processes):
        
                if not process_list[i].is_alive():

                    # fit an updated gaussian process with copies of the output lists
                    gaussian_process.fit(list(output_X), list(output_y))

                    # then start a new process
                    process_list[i] = Process(target=sample_callback, args = (X_eval, output_X, output_Y, lock, i, function,
                        acq_parallel, deepcopy(gaussian_process), np.array(X_eval), np.array(output_y), bounds, find_min,
                        acq_func, penalizer, local_L))
                    process_list[i].start()
                    print(f'Process {i +1} restarted')
                    iteration += 1


    return list(output_X), list(output_y)

## Testing of implementations so far

In [32]:
function = branin
gp_params = None
X_tested = []
y_tested = []
bounds = np.array([[-5, 10], [0, 15]])
n_params = bounds.shape[0]
n_init = 20
X_init = None
    
if X_init is None:
    X_init = np.random.uniform(bounds[:,0], bounds[:,1], (n_init, bounds.shape[0]))
    
for X in X_init:
    X_tested.append(X)
    y_tested.append(function(X))
    
X_np = np.array(X_tested)
y_np = np.array(y_tested)
    
# creating the gaussian process
if gp_params is not None:
    gaussian_process = gp.GaussianProcessRegressor(**args) 
    
else:
    kernel = Matern()
    gaussian_process = gp.GaussianProcessRegressor(
        kernel = kernel, n_restarts_optimizer = 10, normalize_y = True)
    
        
gaussian_process.fit(X_tested, y_tested)
        
X_eval = np.array([[2, 2], [6, 6], [-4, 1], [-3, 12]])
sample_next_parallel(acq_parallel, gaussian_process, X_eval, y_np, bounds)

array([3.22062554, 1.65804109])

# Starting the program

In [ ]:
X_tested, y_tested = bayesian_optimization(200, branin, bounds = np.array([[-5, 10], [0, 15]]), n_init = 16, acq_func = UCB)

best_value = np.min(y_tested)
best_iter = np.argmin(y_tested)
best_X = X_tested[best_iter]
print(best_X, best_value, best_iter)


In [27]:
X = [1,2,3]
Y = np.array(X)
X[1] = 4
Y

array([1, 2, 3])